In [1]:
from kbastroutils.gnd import GND

from astroquery.mast import Observations
from astropy.io import fits

import os,glob,copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol
The following tasks in the acstools package can be run with TEAL:
          acs2d                  acs_destripe           acs_destripe_plus     
          acsccd                    acscte              acscteforwardmodel    
          acsrej                    acssum                    calacs
PixCteCorr is no longer supported. Please use acscte.


In [2]:
DODOWN = False
PID = 15401
DOWNDIR = '/Users/kbhirombhakdi/_work/_data/{0}/G800L_set/'.format(PID)

x = Observations.query_criteria(proposal_id=PID
                                ,filters=['G800L','F775W']
                                ,obs_collection='HST'
#                                 ,target_name='GD71'
                                ,instrument_name='ACS/WFC'
                               )

if DODOWN:
    Observations.download_products(x['obsid']
                                   ,productSubGroupDescription=['FLT']
                                   ,download_dir=DOWNDIR
                                  )
    
# x

In [3]:
FILES = glob.glob(DOWNDIR+'*/*/*/*.fits')

tmp = []
for i in FILES:
    x = fits.open(i)
    if x[0].header['DETECTOR']=='WFC':
        if x[0].header['SUBARRAY']==False:
            tmp.append(i)

FILES = copy.deepcopy(tmp)
FILES

['/Users/kbhirombhakdi/_work/_data/15401/G800L_set/mastDownload/HST/jdql01jxq/jdql01jxq_flt.fits',
 '/Users/kbhirombhakdi/_work/_data/15401/G800L_set/mastDownload/HST/jdql01jvq/jdql01jvq_flt.fits',
 '/Users/kbhirombhakdi/_work/_data/15401/G800L_set/mastDownload/HST/jdql01jnq/jdql01jnq_flt.fits',
 '/Users/kbhirombhakdi/_work/_data/15401/G800L_set/mastDownload/HST/jdql01jpq/jdql01jpq_flt.fits']

# 1.

In [4]:
XXX = GND(FILES)

XXX.make_pair()

XYD = {1: (2099.84,1024.),
       2: (2099.528661638995, 1024.05013586324)
      }
XXX.make_xyd(XYD)
# XXX.make_xyd(adjust=False,box_size=25,maskin=[0])

XXX.make_photparams(method='aperture',apsize=5,apunit='pix',maskin=None
                    ,sigma=3.,iters=5
                    ,dobkgann=True,bkgann=(20.,25.)
                    )
XXX.make_phot()

CONF = {2: '/Users/kbhirombhakdi/_work/_calib_files/ACS.WFC/ACS.WFC.CHIP2.Cycle13.5.conf',
        1: '/Users/kbhirombhakdi/_work/_calib_files/ACS.WFC/ACS.WFC.CHIP1.Cycle13.5.conf'
       }
XXX.make_conf(CONF)

SENS = {2: '/Users/kbhirombhakdi/_work/_calib_files/ACS.WFC/ACS.WFC.1st.sens.7.fits',
        1: '/Users/kbhirombhakdi/_work/_calib_files/ACS.WFC/ACS.WFC.1st.sens.7.fits'
       }
XXX.make_sens(SENS)

# BKG = '/Users/kbhirombhakdi/_work/_calib_files/WFC3.IR.G141.cal.V4.32/WFC3.IR.G141.sky.V1.0.fits'
XXX.make_bkg(method='median',sigma=1.)

XXX.make_xyoff()
XXX.make_xyref()
XXX.make_trace()
XXX.make_wavelength()

XXX.make_stamp(padx=5,pady=20)

# FLAT = '/Users/kbhirombhakdi/_work/_calib_files/WFC3.IR.G141.cal.V4.32/WFC3.IR.G141.flat.2.fits'
XXX.make_flat(method='uniform',flatfile=None)

# PAMFILE = '/Users/kbhirombhakdi/_work/_calib_files/ir_wfc3_map.fits'
XXX.make_pam(method='uniform',pamfile=None)

# # XXX.make_drz()

XXX.make_clean(method=[True,True,False])

XXX.make_exparams(method='aperture',apsize=5,apunit='pix',maskin=None)
XXX.make_wavebin(method='WW')
XXX.make_apcorr(replace='median')
XXX.make_count(replace=None)
XXX.make_flam()

PATH = os.getcwd()
XXX.save(path=PATH,filename='WR96PID15401G800L_01.pickle')

# 2.

In [5]:
# x = XXX.show(method='meta',dosort=True,output=True)
# x['FILE']

In [6]:
# XXX.show(method='direct',zoom=False,dx=100,dy=100)

In [7]:
# XXX.show(method='grism',traceon=True,zoom=False)

In [8]:
# XXX.show(method='CONF',showconf='long')

In [9]:
# XXX.show(method='CONF',showconf='short')

In [10]:
# XXX.show(method='bkg')

In [11]:
# XXX.show(method='flat')

In [12]:
# XXX.show(method='pam')

In [13]:
# XXX.show(method='WW',normalize=True)

In [14]:
# XXX.show(method='trace',normalize=True)

In [15]:
# XXX.show(method='stamp')

In [16]:
# XXX.show(method='clean',traceon=True,zoom=True)

In [17]:
# XXX.show(method='count',apsize=5,tracefrom='original')

In [18]:
# XXX.show(method='count',apsize=5,tracefrom='clean')

In [19]:
# XXX.show(method='flam',xmin=5500.,xmax=10500.)

# 3.